#### 파이썬을 이용한 리포트 수집

In [ ]:
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
url = "http://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222"
response = requests.get(url)
html = response.content
html = BS(html)

In [ ]:
div = html.find("div", attrs = {"class":"board-type"})
li_list = div.find_all("li")

#### 수집 대상 행 결정

In [ ]:
print(li_list[0])

In [ ]:
print(li_list[1])

In [ ]:
print(li_list[-1])

In [ ]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [ ]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

#### 파싱 전략 수립

In [ ]:
stock = li_list[1].find_all("a")[0].text
print(stock)

In [ ]:
title = li_list[1].find_all("a")[1].text
print(title)

In [ ]:
price = li_list[1].find("div", attrs = {"class":"line3"}).text
display(price)

In [ ]:
price = price.replace('\r', '').replace('\t', '').replace('\n', '')
price = price.split(' ')[1].replace(',', '')[:-1]
display(price)

In [ ]:
opinion = li_list[1].find_all("div", attrs = {"class":"line3"})[1].text
display(opinion)

In [ ]:
opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
display(opinion)

In [ ]:
trading_firm = li_list[1].find_all("div", attrs = {"class":"line3"})[2].text
display(trading_firm)

In [ ]:
date = li_list[1].find_all("div", attrs = {"class":"line3"})[3].text
display(date)

In [ ]:
import pandas as pd
date = pd.to_datetime(date)
display(date)

#### 단일 리포트 파싱

In [ ]:
def parsing_li(li):
    stock = li.find_all("a")[0].text # 종목명
    title = li.find_all("a")[1].text # 리포트 제목
    price = li.find("div", attrs = {"class":"line3"}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all("div", attrs = {"class":"line3"})[1].text # 의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 증권사
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)
    
    return [stock, title, price, opinion, trading_firm, date]

In [ ]:
print(parsing_li(li_list[1]))

#### 한 페이지 내 모든 리포트 수집 및 정제

In [ ]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)
data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [ ]:
display(data.head(10))

In [ ]:
import time
base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

for page_no in range(1, 563):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)    
    html = response.content
    html = BS(html)
    
    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", attrs = {"class":"board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])    

In [ ]:
data.to_csv("../../데이터/종목리포트.txt", sep = "\t", encoding = "euc-kr", index = False)